In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.4.0"

#!import ./config/Settings.cs

using Microsoft.SemanticKernel;
using System.IO;

var kernelBuilder = Microsoft.SemanticKernel.Kernel.CreateBuilder();

// Grab the locally stored credentials from the settings.json file. 
// Name the service as "davinci" — assuming that you're using one of the davinci completion models. 
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernelBuilder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    kernelBuilder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = kernelBuilder.Build();

In [ ]:
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.4.0-alpha"

using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#pragma warning disable SKEXP0054

// Load Bing plugin
var bingApiKey = (await InteractiveKernel.GetPasswordAsync("Please enter your Bing Search API key")).GetClearTextPassword();
var bingConnector = new BingConnector(bingApiKey);

kernel.ImportPluginFromObject(new WebSearchEnginePlugin(bingConnector), "bing");

In [ ]:
var web = kernel.Plugins["bing"]["search"];

var bingResult = await kernel.InvokeAsync(web, new(){ ["query"] = "What's the tallest building in NYC?" });
Console.WriteLine(bingResult);